# Preliminary Steps (Mandatory)

- Create a kernel (or virtual environment, venv)
- Place the CSV file in the /csv folder

In [ ]:
%pip install pandas
%pip install PyYAML
%pip install matplotlib

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

csv_folder = 'csv/'

# Get a list of CSV files in the folder
csv_files = glob.glob(csv_folder + '*.csv')

# Check if any CSV files are found
if csv_files:
    # Load the first CSV file into a pandas DataFrame
    first_csv_file = csv_files[0]
    
    # print("DataFrame loaded successfully.")
    # print(df.head())  # Print the first few rows of the DataFrame
else:
    print("No CSV files found in the folder.")

## Extract bodyparts from Dataframe

In [ ]:
# Load the CSV data into a pandas DataFrame
df = pd.read_csv(first_csv_file, skiprows=1)

head_x = df['head'][1:].astype(float)  # Remove the first element and convert to float for plotting
head_y = df['head.1'][1:].astype(float)  # Remove the first element and convert to float for plotting

nose_x = df['nose'][1:].astype(float)  # Remove the first element and convert to float for plotting
nose_y = df['nose.1'][1:].astype(float)  # Remove the first element and convert to float for plotting

spine1_x = df['spine1'][1:].astype(float)  # Remove the first element and convert to float for plotting
spine1_y = df['spine1.1'][1:].astype(float)  # Remove the first element and convert to float for plotting

spine2_x = df['spine2'][1:].astype(float)  # Remove the first element and convert to float for plotting
spine2_y = df['spine2.1'][1:].astype(float)  # Remove the first element and convert to float for plotting

spine3_x = df['spine3'][1:].astype(float)  # Remove the first element and convert to float for plotting
spine3_y = df['spine3.1'][1:].astype(float)  # Remove the first element and convert to float for plotting

spine4_x = df['spine4'][1:].astype(float)  # Remove the first element and convert to float for plotting
spine4_y = df['spine4.1'][1:].astype(float)  # Remove the first element and convert to float for plotting

# Line Plot (Optional)

In [ ]:
# Generate indices starting from 1
indices = range(1, len(head_x) + 1)

# Plot head_x and head_y
plt.plot(indices, head_x, label='head_x')
plt.plot(indices, head_y, label='head_y')

# Add labels and legend
plt.xlabel('Index')
plt.ylabel('Pixel Coordinates')
plt.legend()

# Show the plot
plt.show()

# Scatter Plot (Optional)

In [ ]:
# Decrease the size of the dots
dot_size = 5  # Adjust the size as needed

# Plot x vs y axis
plt.scatter(head_x, head_y, s=dot_size, label='Head')
plt.scatter(spine1_x, spine1_y, s=dot_size, label='Spine1')
plt.scatter(spine2_x, spine2_y, s=dot_size, label='Spine2')
plt.scatter(spine3_x, spine3_y, s=dot_size, label='Spine3')
plt.scatter(spine4_x, spine4_y, s=dot_size, label='Spine4')
plt.scatter(nose_x, nose_y, s=dot_size, label='Nose')

# Add labels
plt.xlabel('X')
plt.ylabel('Y')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Show the plot
plt.show()